In [94]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [95]:
df_UK= pd.read_csv("df_UK.csv" ,encoding='unicode_escape')


In [96]:
df_UK

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,United Kingdom,2020/1/1,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,United Kingdom,2020/1/2,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,United Kingdom,2020/1/3,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,United Kingdom,2020/1/4,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,United Kingdom,2020/1/5,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,United Kingdom,2021/9/24,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,1.0,0.0,0.0,3.0,5.0,2.0,0.017889
1595,United Kingdom,2021/9/25,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017835
1596,United Kingdom,2021/9/26,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017767
1597,United Kingdom,2021/9/27,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017685


In [97]:
#shift 21days mortality
df_UK_mortality=df_UK["mortality"]
df_UK_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
          ...   
1594    0.017889
1595    0.017835
1596    0.017767
1597    0.017685
1598    0.000000
Name: mortality, Length: 1577, dtype: float64

In [98]:
#删除后21天
df_UK.drop(df_UK.index[1577:1599], inplace=True)
df_UK

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,United Kingdom,2020/1/1,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,United Kingdom,2020/1/2,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,United Kingdom,2020/1/3,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,United Kingdom,2020/1/4,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,United Kingdom,2020/1/5,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572,United Kingdom,2021/9/2,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,1.0,0.0,0.0,3.0,5.0,2.0,0.019368
1573,United Kingdom,2021/9/3,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,1.0,0.0,0.0,3.0,5.0,2.0,0.019267
1574,United Kingdom,2021/9/4,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,1.0,0.0,0.0,3.0,5.0,2.0,0.019183
1575,United Kingdom,2021/9/5,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,1.0,0.0,0.0,3.0,5.0,2.0,0.019092


In [99]:
df_UK1=df_UK.join(df_UK_mortality,how="left",rsuffix="after21days")
df_UK1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,United Kingdom,2020/1/1,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,United Kingdom,2020/1/2,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,United Kingdom,2020/1/3,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,United Kingdom,2020/1/4,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,United Kingdom,2020/1/5,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572,United Kingdom,2021/9/2,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,1.0,0.0,0.0,3.0,5.0,2.0,0.019368,0.019368
1573,United Kingdom,2021/9/3,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,1.0,0.0,0.0,3.0,5.0,2.0,0.019267,0.019267
1574,United Kingdom,2021/9/4,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,1.0,0.0,0.0,3.0,5.0,2.0,0.019183,0.019183
1575,United Kingdom,2021/9/5,67886011,83%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,1.0,0.0,0.0,3.0,5.0,2.0,0.019092,0.019092


In [75]:
df_UK1.to_csv("df_UK_after21days",index=False)